In [31]:
from scipy.stats import rankdata
from dateutil import parser
import numpy as np
import numpy.linalg as la  
import pandas as pd
from datetime import datetime
import scipy.stats as stats
import matplotlib.pyplot as plt

In [24]:
# amount	ths	amount	成交额
# close_hfq	ths	close	收盘价
# open_hfq	ths	open	开盘价
# high_hfq	ths	high	最高价
# low_hfq	ths	low	最低价
# volume	ths	volume	成交量
# totalShares	ths	totalShares	总股本
# floatshare	ths	floatSharesOfAShares	A股流通股本
# cap	ths	totalCapital	总市值
# transactionAmount	ths	transactionAmount	成交笔数
# turnover	ths	turnoverRatio	换手率
# PE_ttm	ths	pe_ttm	市盈率（TTM）
# vwap	ths	avgPrice	市值加权均价
# change	ths	changeRatio	涨跌幅
# pb_lf	ths	pb	市净率
# ps_ttm	ths	ps	市销率

Open = pd.read_csv("open_hfq.csv",parse_dates=[0], index_col=0).fillna(value=0).reset_index().drop(columns = ['Date'])
volume = pd.read_csv("volume.csv", parse_dates=[0], index_col=0).fillna(value=0).reset_index().drop(columns = ['Date'])
close = pd.read_csv("close_hfq.csv",parse_dates=[0], index_col=0).fillna(value=0).reset_index().drop(columns = ['Date'])
high = pd.read_csv("high_hfq.csv",parse_dates=[0], index_col=0).fillna(value=0).reset_index().drop(columns = ['Date'])
low = pd.read_csv("low_hfq.csv",parse_dates=[0], index_col=0).fillna(value=0).reset_index().drop(columns = ['Date'])
vwap = pd.read_csv("vwap.csv",parse_dates=[0], index_col=0).fillna(value=0).reset_index().drop(columns = ['Date'])
cap = pd.read_csv("cap.csv",parse_dates=[0], index_col=0).fillna(value=0).reset_index().drop(columns = ['Date'])
turnover  = pd.read_csv("turnover.csv",parse_dates=[0], index_col=0).fillna(value=0).reset_index().drop(columns = ['Date'])
returns = pd.read_csv("change_ratio.csv",parse_dates=[0], index_col=0).fillna(value=0).reset_index().drop(columns = ['Date'])
pb_lf = pd.read_csv('pb_lf.csv', parse_dates=[0], index_col=0).reset_index().drop(columns=['Date'])
pe_ttm = pd.read_csv('PE_ttm.csv', parse_dates=[0], index_col=0).reset_index().drop(columns=['Date'])
ps_ttm = pd.read_csv('ps_ttm.csv', parse_dates=[0], index_col=0).reset_index().drop(columns=['Date'])
amount = pd.read_csv('amount.csv', parse_dates=[0], index_col=0).reset_index().drop(columns=['Date'])

In [30]:

## 内置函数
# # 行业中性化
# def neutral(data,ind):
#     stocks = list(data.index)
#     data_ = pd.DataFrame(ind.Data,index=['INDUSTRY'],columns=ind.Codes).T
#     data_med = pd.get_dummies(data_,columns=['INDUSTRY'])
#     X= np.array(data_med)
#     y = data.values
#     beta_ols = la.inv(X.T.dot(X)).dot(X.T).dot(y)
#     residual = y - X.dot(beta_ols)
#     return residual
# def IndNeutralize(vwap,ind):
#     vwap_ = vwap.fillna(value = 0)
#     ind = w.wss(stocks, "industry_citic","unit=1;tradeDate="+ind.Times[0].strftime("%Y%m%d")+";industryType=1")
#     for i in range(len(vwap_)):
#         vwap_.iloc[i] = neutral(vwap_.iloc[i],ind)
#     return vwap_
# 移动求和
def ts_sum(df, window):
    return df.rolling(window).sum()
# 移动平均
def sma(df, window):
    return df.rolling(window).mean()
# 移动标准差
def stddev(df, window):
    return df.rolling(window).std()
# 移动相关系数
def correlation(x, y, window):
    return x.rolling(window).corr(y)
# 移动协方差
def covariance(x, y, window):
    return x.rolling(window).cov(y)
# 在过去d天的时序排名
def rolling_rank(na):
    return rankdata(na)[-1]
def ts_rank(df,window):
    return df.rolling(window).apply(rolling_rank)
# 过去d天的时序乘积
def rolling_prod(na):
    return np.prod(na)
def product(df, window):
    return df.rolling(window).apply(rolling_prod)
# 过去d天最小值
def ts_min(df, window):
    return df.rolling(window).min()
# 过去d天最大值
def ts_max(df, window):
    return df.rolling(window).max()
# 当天取值减去d天前的值
def delta(df, period):
    return df.diff(period)
# d天前的值，滞后值
def delay(df, period):
    return df.shift(period)
# 截面数据排序，输出boolean值
def rank(df):
    return df.rank(pct=True,axis=1)
# 缩放时间序列，使其和为1
def scale(df, k=1):
    return df.mul(k).div(np.abs(df).sum())
# 过去d天最大值的位置
def ts_argmax(df, window):
    return df.rolling(window).apply(np.argmax) + 1
# 过去d天最小值的位置
def ts_argmin(df, window):
    return df.rolling(window).apply(np.argmin) + 1
# 线性衰减的移动平均加权
def decay_linear(df, period):
    if df.isnull().values.any():
        df.fillna(method='ffill', inplace=True)
        df.fillna(method='bfill', inplace=True)
        df.fillna(value=0, inplace=True)
    na_lwma = np.zeros_like(df) # 生成与df大小相同的零数组
    na_lwma[:period, :] = df.iloc[:period, :] # 赋前period项的值
    na_series = df.iloc[:,:].values
    # 计算加权系数
    divisor = period * (period + 1) / 2
    y = (np.arange(period) + 1) * 1.0 / divisor
    # 从第period项开始计算数值
    for row in range(period - 1, df.shape[0]):
        x = na_series[row - period + 1: row + 1, :]
        na_lwma[row, :] = (np.dot(x.T, y))
    return pd.DataFrame(na_lwma,index = df.index,columns = df.columns)

## 因子函数
def alpha1(close,returns):
    x = close
    x[returns < 0] = stddev(returns,20)
    alpha = rank(ts_argmax(x ** 2, 5))-0.5
    return alpha.fillna(value = 0)
def alpha2(Open,close,volume):
    r1 = rank(delta(np.log(volume), 2))
    r2 = rank((close - Open) / Open)
    alpha = -1 * correlation(r1,r2,6)
    return alpha.fillna(value = 0)
def alpha3(Open,volume):
    r1 = rank(Open)
    r2 = rank(volume)
    alpha = -1 * correlation(r1,r2,10)
    return alpha.replace([-np.inf, np.inf], 0).fillna(value = 0)
def alpha4(low):
    r = rank(low)
    alpha = -1 * ts_rank(r,9)
    return alpha.fillna(value = 0)
def alpha5(Open,vwap,close):
    alpha = (rank((Open - (ts_sum(vwap, 10) / 10))) * (-1 * abs(rank((close - vwap)))))
    return alpha.fillna(value = 0)
def alpha6(Open, volume):
    alpha = -1 * correlation(Open, volume, 10)
    return alpha.replace([-np.inf, np.inf], 0).fillna(value = 0)
def alpha7(volume,close):
    adv20 = sma(volume, 20)
    alpha = -1 * ts_rank(abs(delta(close, 7)), 60) * np.sign(delta(close, 7))
    alpha[adv20 >= volume] = -1
    return alpha.fillna(value = 0)
def alpha8(Open,returns):
    x1 = (ts_sum(Open, 5) * ts_sum(returns, 5))
    x2 = delay((ts_sum(Open, 5) * ts_sum(returns, 5)), 10)
    alpha = -1 * rank(x1-x2)
    return alpha.fillna(value = 0)
def alpha9(close):
    delta_close = delta(close, 1)
    x1 = ts_min(delta_close, 5) > 0
    x2 = ts_max(delta_close, 5) < 0
    alpha = -1 * delta_close
    alpha[x1 | x2] = delta_close
    return alpha.fillna(value = 0)
def alpha10(close):
    delta_close = delta(close, 1)
    x1 = ts_min(delta_close, 4) > 0
    x2 = ts_max(delta_close, 4) < 0
    x = -1 * delta_close
    x[x1 | x2] = delta_close
    alpha = rank(x)
    return alpha.fillna(value = 0)
def alpha11(vwap,close,volume):
    x1 = rank(ts_max((vwap - close), 3))
    x2 = rank(ts_min((vwap - close), 3))
    x3 = rank(delta(volume, 3))
    alpha = (x1 + x2) * x3
    return alpha.fillna(value = 0)
def alpha12(volume,close):
    alpha = np.sign(delta(volume, 1)) * (-1 * delta(close, 1))
    return alpha.fillna(value = 0)
def alpha13(volume,close):
    alpha = -1 * rank(covariance(rank(close), rank(volume), 5))
    return alpha.fillna(value = 0)
def alpha14(Open,volume,returns):
    x1 = correlation(Open, volume, 10).replace([-np.inf, np.inf], 0).fillna(value=0)
    x2 = -1 * rank(delta(returns, 3))
    alpha = x1 * x2
    return alpha.fillna(value = 0)
def alpha15(high,volume):
    x1 = correlation(rank(high), rank(volume), 3).replace([-np.inf, np.inf], 0).fillna(value=0)
    alpha = -1 * ts_sum(rank(x1), 3)
    return alpha.fillna(value = 0)
def alpha16(high,volume):
    alpha = -1 * rank(covariance(rank(high), rank(volume), 5))
    return alpha.fillna(value = 0)
def alpha17(volume,close):
    adv20 = sma(volume, 20)
    x1 = rank(ts_rank(close, 10))
    x2 = rank(delta(delta(close, 1), 1))
    x3 = rank(ts_rank((volume / adv20), 5))
    alpha = -1 * (x1 * x2 * x3)
    return alpha.fillna(value = 0)
def alpha18(close,Open):
    x = correlation(close, Open, 10).replace([-np.inf, np.inf], 0).fillna(value=0)
    alpha = -1 * (rank((stddev(abs((close - Open)), 5) + (close - Open)) + x))
    return alpha.fillna(value = 0)
def alpha19(close,returns):
    x1 = (-1 * np.sign((close - delay(close, 7)) + delta(close, 7)))
    x2 = (1 + rank(1 + ts_sum(returns, 250)))
    alpha = x1 * x2
    return alpha.fillna(value = 0)
def alpha20(Open,high,close,low):
    alpha = -1 * (rank(Open - delay(high, 1)) * rank(Open - delay(close, 1)) * rank(Open - delay(low, 1)))
    return alpha.fillna(value = 0)
def alpha21(volume,close):
    x1 = sma(close, 8) + stddev(close, 8) < sma(close, 2)
    x2 = sma(close, 8) - stddev(close, 8) > sma(close, 2)
    x3 = sma(volume, 20) / volume < 1
    alpha = pd.DataFrame(np.ones_like(close), index = close.index,columns = close.columns)
    alpha[x1 | x3] = -1 * alpha
    return alpha
def alpha22(high,volume,close):
    x = correlation(high, volume, 5).replace([-np.inf, np.inf], 0).fillna(value=0)
    alpha = -1 * delta(x, 5) * rank(stddev(close, 20))
    return alpha.fillna(value = 0)
def alpha23(high,close):
    x = sma(high, 20) < high
    alpha = pd.DataFrame(np.zeros_like(close),index = close.index,columns = close.columns)
    alpha[x] = -1 * delta(high, 2).fillna(value = 0)
    return alpha
def alpha24(close):
    x = delta(sma(close, 100), 100) / delay(close, 100) <= 0.05
    alpha = -1 * delta(close, 3)
    alpha[x] = -1 * (close - ts_min(close, 100))
    return alpha.fillna(value = 0)
def alpha25(volume,returns,vwap,high,close):
    adv20 = sma(volume, 20)
    alpha = rank((((-1 * returns) * adv20) * vwap) * (high - close))
    return alpha.fillna(value = 0)
def alpha26(volume,high):
    x = correlation(ts_rank(volume, 5), ts_rank(high, 5), 5).replace([-np.inf, np.inf], 0).fillna(value=0)
    alpha = -1 * ts_max(x, 3)
    return alpha.fillna(value = 0)
def alpha27(volume,vwap):
    alpha = rank((sma(correlation(rank(volume), rank(vwap), 6), 2) / 2.0))
    alpha[alpha > 0.5] = -1
    alpha[alpha <= 0.5] = 1
    return alpha.fillna(value = 0)
def alpha28(volume,high,low,close):
    adv20 = sma(volume, 20)
    x = correlation(adv20, low, 5).replace([-np.inf, np.inf], 0).fillna(value=0)
    alpha = scale(((x + ((high + low) / 2)) - close))
    return alpha.fillna(value = 0)
def alpha29(close,returns):
    x1 = ts_min(rank(rank(scale(np.log(ts_sum(rank(rank(-1 * rank(delta((close - 1), 5)))), 2))))), 5)
    x2 = ts_rank(delay((-1 * returns), 6), 5)
    alpha = x1 + x2
    return alpha.fillna(value = 0)
def alpha30(close,volume):
    delta_close = delta(close, 1)
    x = np.sign(delta_close) + np.sign(delay(delta_close, 1)) + np.sign(delay(delta_close, 2))
    alpha = ((1.0 - rank(x)) * ts_sum(volume, 5)) / ts_sum(volume, 20)
    return alpha.fillna(value = 0)
def alpha31(close,low,volume):
    adv20 = sma(volume,20)
    x1 = rank(rank(rank(decay_linear((-1 * rank(rank(delta(close, 10)))), 10))))
    x2 = rank((-1 * delta(close, 3)))
    x3 = np.sign(scale(correlation(adv20, low, 12).replace([-np.inf, np.inf], 0).fillna(value=0)))
    alpha = x1 + x2 + x3
    return alpha.fillna(value = 0)
def alpha32(close,vwap):
    x = correlation(vwap, delay(close, 5),230).replace([-np.inf, np.inf], 0).fillna(value=0)
    alpha = scale(((sma(close, 7)) - close)) + 20 * scale(x)
    return alpha.fillna(value = 0)
def alpha33(Open,close):
    alpha = rank(-1 + (Open / close))
    return alpha
def alpha34(close,returns):
    x = (stddev(returns, 2) / stddev(returns, 5)).fillna(value = 0)
    alpha = rank(2 - rank(x) - rank(delta(close, 1)))
    return alpha.fillna(value = 0)
def alpha35(volume,close,high,low,returns):
    x1 = ts_rank(volume, 32)
    x2 = 1 - ts_rank(close + high - low, 16)
    x3 = 1 - ts_rank(returns, 32)
    alpha = (x1 * x2 * x3).fillna(value = 0)
    return alpha
def alpha36(Open,close,volume,returns,vwap):
    adv20 = sma(volume, 20)
    x1 = 2.21 * rank(correlation((close - Open), delay(volume, 1), 15))
    x2 = 0.7 * rank((Open- close))
    x3 = 0.73 * rank(ts_rank(delay((-1 * returns), 6), 5))
    x4 = rank(abs(correlation(vwap,adv20, 6)))
    x5 = 0.6 * rank((sma(close, 200) - Open) * (close - Open))
    alpha = x1 + x2 + x3 + x4 + x5
    return alpha.fillna(value = 0)
def alpha37(Open,close):
    alpha = rank(correlation(delay(Open - close, 1), close, 200)) + rank(Open - close)
    return alpha
def alpha38(close,Open):
    x = (close / Open).replace([-np.inf, np.inf], 0).fillna(value=0)
    alpha = -1 * rank(ts_rank(Open, 10)) * rank(x)
    return alpha.fillna(value = 0)
def alpha39(volume,close,returns):
    adv20 = sma(volume, 20)
    x = -1 * rank(delta(close, 7)) * (1 - rank(decay_linear((volume / adv20), 9)))
    alpha = x *(1 + rank(ts_sum(returns, 250)))
    return alpha.fillna(value = 0)
def alpha40(high,volume):
    alpha = -1 * rank(stddev(high, 10)) * correlation(high, volume, 10)
    return alpha.fillna(value = 0)
def alpha41(high,low,vwap):
    alpha = pow((high * low),0.5) - vwap
    return alpha
def alpha42(vwap,close):
    alpha = rank((vwap - close)) / rank((vwap + close))
    return alpha
def alpha43(volume,close):
    adv20 = sma(volume, 20)
    alpha = ts_rank(volume / adv20, 20) * ts_rank((-1 * delta(close, 7)), 8)
    return alpha.fillna(value = 0)
def alpha44(high,volume):
    alpha = -1 *correlation(high, rank(volume), 5).replace([-np.inf, np.inf], 0).fillna(value=0)
    return alpha
def alpha45(close,volume):
    x = correlation(close, volume, 2).replace([-np.inf, np.inf], 0).fillna(value=0)
    alpha = -1 * (rank(sma(delay(close, 5), 20)) * x * rank(correlation(ts_sum(close, 5), ts_sum(close, 20), 2)))
    return alpha.fillna(value = 0)
def alpha46(close):
    x = ((delay(close, 20) - delay(close, 10)) / 10) - ((delay(close, 10) - close) / 10)
    alpha = (-1 * (close - delay(close, 1)))
    alpha[x < 0] = 1
    alpha[x > 0.25] = -1
    return alpha.fillna(value = 0)
def alpha47(volume,close,high,vwap):
    adv20 = sma(volume, 20)
    alpha = ((rank((1 / close)) * volume) / adv20) * ((high * rank((high - close))) / sma(high, 5)) - rank((vwap - delay(vwap, 5)))
    return alpha.fillna(value = 0)
def alpha49(close):
    x = (((delay(close, 20) - delay(close, 10)) / 10) - ((delay(close, 10) - close) / 10))
    alpha = (-1 * delta(close,1))
    alpha[x < -0.1] = 1
    return alpha.fillna(value = 0)
def alpha50(volume,vwap):
    alpha = -1 * ts_max(rank(correlation(rank(volume), rank(vwap), 5)), 5)
    return alpha.fillna(value = 0)
def alpha51(close):
    inner = (((delay(close, 20) - delay(close, 10)) / 10) - ((delay(close, 10) - close) / 10))
    alpha = (-1 * delta(close,1))
    alpha[inner < -0.05] = 1
    return alpha.fillna(value = 0)
def alpha52(returns,volume,low):
    x = rank(((ts_sum(returns, 240) - ts_sum(returns, 20)) / 220))
    alpha = -1 * delta(ts_min(low, 5), 5) * x * ts_rank(volume, 5)
    return alpha.fillna(value = 0)
def alpha53(close,high,low):
    alpha = -1 * delta((((close - low) - (high - close)) / (close - low).replace(0, 0.0001)), 9)
    return alpha.fillna(value = 0)
def alpha54(Open,close,high,low):
    x = (low - high).replace(0, -0.0001)
    alpha = -1 * (low - close) * (Open ** 5) / (x * (close ** 5))
    return alpha
def alpha55(high,low,close,volume):
    x = (close - ts_min(low, 12)) / (ts_max(high, 12) - ts_min(low, 12)).replace(0, 0.0001)
    alpha = -1 * correlation(rank(x), rank(volume), 6).replace([-np.inf, np.inf], 0).fillna(value=0)
    return alpha
def alpha56(returns,cap):
    alpha = 0 - (1 * (rank((sma(returns, 10) / sma(sma(returns, 2), 3))) * rank((returns * cap))))
    return alpha.fillna(value = 0)
def alpha57(close,vwap):
    alpha = 0 - 1 * ((close - vwap) / decay_linear(rank(ts_argmax(close, 30)), 2))
    return alpha.fillna(value = 0)
def alpha60(close,high,low,volume):
    x = ((close - low) - (high - close)) * volume / (high - low).replace(0, 0.0001)
    alpha = - ((2 * scale(rank(x))) - scale(rank(ts_argmax(close, 10))))
    return alpha.fillna(value = 0)
def alpha61(volume,vwap):
    adv180 = sma(volume, 180)
    alpha = rank((vwap - ts_min(vwap, 16))) < rank(correlation(vwap, adv180, 18))
    return alpha
def alpha62(volume,high,low,Open,vwap):
    adv20 = sma(volume, 20)
    x1 = rank(correlation(vwap, ts_sum(adv20, 22), 10))
    x2 =  rank(((rank(Open) + rank(Open)) < (rank(((high + low) / 2)) + rank(high))))
    alpha = x1 < x2
    return alpha*-1
def alpha64(high,low,Open,volume,vwap):
    adv120 = sma(volume, 120)
    x1 = rank(correlation(ts_sum(((Open * 0.178404) + (low * (1 - 0.178404))), 13),ts_sum(adv120, 13), 17))
    x2 = rank(delta(((((high + low) / 2) * 0.178404) + (vwap * (1 -0.178404))), 3.69741))
    alpha = x1 < x2
    return alpha*-1
def alpha65(volume,vwap,Open):
    adv60 = sma(volume, 60)
    x1 = rank(correlation(((Open * 0.00817205) + (vwap * (1 - 0.00817205))), ts_sum(adv60,9), 6))
    x2 = rank((Open - ts_min(Open, 14)))
    alpha = x1 < x2
    return alpha*-1
def alpha66(vwap,low,Open,high):
    x1 = rank(decay_linear(delta(vwap, 4), 7))
    x2 = (((low* 0.96633) + (low * (1 - 0.96633))) - vwap) / (Open - ((high + low) / 2))
    alpha = (x1 + ts_rank(decay_linear(x2, 11), 7)) * -1
    return alpha.fillna(value = 0)
def alpha68(volume,high,close,low):
    adv15 = sma(volume, 15)
    x1 = ts_rank(correlation(rank(high), rank(adv15), 9), 14)
    x2 = rank(delta(((close * 0.518371) + (low * (1 - 0.518371))), 1.06157))
    alpha = x1 < x2
    return alpha*-1
def alpha71(volume,close,low,Open,vwap):
    adv180 = sma(volume, 180)
    x1 = ts_rank(decay_linear(correlation(ts_rank(close, 3), ts_rank(adv180,12), 18), 4), 16)
    x2 = ts_rank(decay_linear((rank(((low + Open) - (vwap + vwap))).pow(2)), 16), 4)
    alpha = x1
    alpha[x1 < x2] = x2
    return alpha.fillna(value = 0)
def alpha72(volume,high,low,vwap):
    adv40 = sma(volume, 40)
    x1 = rank(decay_linear(correlation(((high + low) / 2), adv40, 9), 10))
    x2 = rank(decay_linear(correlation(ts_rank(vwap, 4), ts_rank(volume, 19), 7), 3))
    alpha = (x1 / x2.replace(0, 0.0001)).fillna(value = 0)
    return alpha
def alpha73(vwap,Open,low):
    x1 = rank(decay_linear(delta(vwap, 5), 3))
    x2 = delta(((Open * 0.147155) + (low * (1 - 0.147155))), 2) / ((Open *0.147155) + (low * (1 - 0.147155)))
    x3 = ts_rank(decay_linear((x2 * -1), 3), 17)
    alpha = x1
    alpha[x1 < x3] = x3
    return -1 * alpha.fillna(value = 0)
def alpha74(volume,close,high,vwap):
    adv30 = sma(volume, 30)
    x1 = rank(correlation(close, ts_sum(adv30, 37), 15))
    x2 = rank(correlation(rank(((high * 0.0261661) + (vwap * (1 - 0.0261661)))), rank(volume), 11))
    alpha = x1 < x2
    return alpha*-1
def alpha75(volume,vwap,low):
    adv50 = sma(volume, 50)
    alpha = rank(correlation(vwap, volume, 4)) < rank(correlation(rank(low), rank(adv50), 12))
    return alpha
def alpha77(volume,high,low,vwap):
    adv40 = sma(volume, 40)
    x1 = rank(decay_linear(((((high + low) / 2) + high) - (vwap + high)), 20))
    x2 = rank(decay_linear(correlation(((high + low) / 2), adv40, 3), 6))
    alpha = x1
    alpha[x1 > x2] = x2
    return alpha.fillna(value = 0)
def alpha78(volume,low,vwap):
    adv40 = sma(volume, 40)
    x1 = rank(correlation(ts_sum(((low * 0.352233) + (vwap * (1 - 0.352233))), 20), ts_sum(adv40, 20),7))
    x2 = rank(correlation(rank(vwap), rank(volume), 6))
    alpha = x1.pow(x2)
    return alpha.fillna(value = 0)
def alpha81(volume,vwap):
    adv10 = sma(volume, 10)
    x1 = rank(np.log(product(rank((rank(correlation(vwap, ts_sum(adv10, 50), 8)).pow(4))), 15)))
    x2 = rank(correlation(rank(vwap), rank(volume), 5))
    alpha = x1 < x2
    return alpha*-1
def alpha83(high,low,close,volume):
    x = rank(delay(((high - low) / (ts_sum(close, 5) / 5)), 2)) * rank(rank(volume))
    alpha = x / (((high - low) / (ts_sum(close, 5) / 5)) / (vwap - close))
    return alpha.fillna(value = 0)
def alpha84(vwap,close):
    alpha = pow(ts_rank((vwap - ts_max(vwap, 15)), 21), delta(close,5))
    return alpha.fillna(value = 0)
def alpha85(volume,high,close,low):
    adv30 = sma(volume, 30)
    x1 = rank(correlation(((high * 0.876703) + (close * (1 - 0.876703))), adv30,10))
    alpha = x1.pow(rank(correlation(ts_rank(((high + low) / 2), 4), ts_rank(volume, 10), 7)))
    return alpha.fillna(value = 0)
def alpha86(volume,close,Open,vwap):
    adv20 = sma(volume, 20)
    x1 = ts_rank(correlation(close, sma(adv20, 15), 6), 20)
    x2 = rank(((Open+ close) - (vwap + Open)))
    alpha = x1 < x2
    return alpha*-1
def alpha88(volume,Open,low,high,close):
    adv60 = sma(volume, 60)
    x1 = rank(decay_linear(((rank(Open) + rank(low)) - (rank(high) + rank(close))),8))
    x2 = ts_rank(decay_linear(correlation(ts_rank(close, 8), ts_rank(adv60,21), 8), 7), 3)
    alpha = x1
    alpha[x1 > x2] = x2
    return alpha.fillna(value = 0)
def alpha92(volume,high,low,close,Open):
    adv30 = sma(volume, 30)
    x1 = ts_rank(decay_linear(((((high + low) / 2) + close) < (low + Open)), 15), 19)
    x2 = ts_rank(decay_linear(correlation(rank(low), rank(adv30), 8), 7),7)
    alpha = x1
    alpha[x1 > x2] = x2
    return alpha.fillna(value = 0)
def alpha94(volume,vwap):
    adv60 = sma(volume, 60)
    x = rank((vwap - ts_min(vwap, 12)))
    alpha = x.pow(ts_rank(correlation(ts_rank(vwap,20), ts_rank(adv60, 4), 18), 3))* -1
    return alpha.fillna(value = 0)
def alpha95(volume,high,low,Open):
    adv40 = sma(volume, 40)
    x = ts_rank((rank(correlation(sma(((high + low)/ 2), 19), sma(adv40, 19), 13)).pow(5)), 12)
    alpha = rank((Open - ts_min(Open, 12))) < x
    return alpha.fillna(value = 0)
def alpha96(volume,vwap,close):
    adv60 = sma(volume, 60)
    x1 = ts_rank(decay_linear(correlation(rank(vwap), rank(volume), 4),4), 8)
    x2 = ts_rank(decay_linear(ts_argmax(correlation(ts_rank(close, 7),ts_rank(adv60, 4), 4), 13), 14), 13)
    alpha = x1
    alpha[x1 < x2] = x2
    return alpha.fillna(value = 0)
def alpha98(volume,Open,vwap):
    adv5 = sma(volume, 5)
    adv15 = sma(volume, 15)
    x1 = rank(decay_linear(correlation(vwap, sma(adv5, 26), 5), 7))
    alpha = x1 - rank(decay_linear(ts_rank(ts_argmin(correlation(rank(Open), rank(adv15), 21), 9),7), 8))
    return alpha.fillna(value = 0)
def alpha99(volume,high,low):
    adv60 = sma(volume, 60)
    x1 = rank(correlation(ts_sum(((high + low) / 2), 20), ts_sum(adv60,20), 9))
    x2 = rank(correlation(low, volume, 6))
    alpha = x1 < x2
    return alpha*-1
def alpha101(close,Open,high,low):
    alpha = (close - Open) / ((high - low) + 0.001)
    return alpha

def BP(pb_lf):   #BP是市净率pb_lf的倒数，来自华泰研报
    # 定义一个函数，用于将一个数取倒数
    def reciprocal(x):
        if x == 0:
            return 0
        else:
            return 1 / x
    # 使用apply()函数将函数应用到每个单元格中
    BP = pb_lf.applymap(reciprocal)
    return BP

def EP(pe_ttm):  #EP是市盈率pe_ttm的倒数，来自华泰研报
    def reciprocal(x):
        if x == 0:
            return 0
        else:
            return 1 / x
    EP = pe_ttm.applymap(reciprocal)
    return EP

def SP(ps_ttm):  #SP是市销率ps_ttm的倒数，来自华泰研报
    def reciprocal(x):
        if x == 0:
            return 0
        else:
            return 1 / x
    SP = ps_ttm.applymap(reciprocal)
    return SP

def daoshu(dataf):
    return 0 if dataf==0 else 1/dataf

def std_60days(close):  # 个股最近 3个月内日收益率序列的标准差,华泰研报
    window = 60
    returns = close.pct_change()  #计算收益率
    def stddev(df, window):  # 移动标准差
        return df.rolling(window).std()
    std_60days = stddev(returns, window)
    return std_60days


def turnover14(turnover):
    turnover_14 = sma(turnover, 14)
    return turnover_14

def turnover28(turnover):
    turnover_28 = sma(turnover, 28)
    return turnover_28

def turnover14predicted(turnover_14,turnover_28):
    df_increase=(turnover_14-turnover_28)/turnover_28
    return turnover_14*(1+df_increase)

def turnoveradvance01(turnover,pctchange,section_return,section_info,d=5,m=30,H=15):
    # d,m,H为参数，风险均值窗口d，收益加权窗口m，H一般为m的一半，作为半衰期
    # turnover为换手率的原始dataframe
    # pctchange为涨跌幅的原始dataframe
    # section_return为对应行业的涨跌幅的dataframe，行为行业指数，列为日期
    # section_info为股票及其对应的行业的对照表dataframe，列名为'industrycode'
    def weight_omega(m,j,H):
        aa=[2**(-i/H) for i in range(1,m+1)]
        return 2**(-(m-j+1)/H)/sum(aa)

    def risk_today(origin,d):
        return origin.rolling(d).mean()-origin
    
    weight_value=np.array([weight_omega(m,j,H) for j in range(1,m+1)])
    
    stocklist=turnover.columns.intersection(section_info[section_info['industrycode']!='0'].index)
    turnover=turnover[stocklist]
    pctchange=pctchange[stocklist]
    section_return=section_return.dropna(axis=1, how='any')
    search=section_info.applymap(lambda x:x in section_return.columns)
    selected=section_info[search.industrycode].index
    stocklist=turnover.columns.intersection(selected)
    turnover=turnover[stocklist]
    pctchange=pctchange[stocklist]
    
    df_exceed=pd.DataFrame(index=pctchange.index,columns=pctchange.columns)
    for ii in pctchange.columns:
        df_exceed[ii]=pctchange[ii]-section_return[section_info.loc[ii].values[0]]
    df_risk=pd.DataFrame(index=pctchange.index,columns=pctchange.columns)
    for ii in pctchange.columns:
        df_risk[ii]=risk_today(turnover[[ii]],d)
    df_riskturnover=df_risk*df_exceed
    df_riskturnover=df_riskturnover.iloc[d:,:]
    df_final=pd.DataFrame(index=df_riskturnover.index[m:],columns=df_riskturnover.columns)
    for jj in range(len(df_final.index)):
        df_final.iloc[jj,:]=weight_value@df_riskturnover.iloc[jj:(jj+m),:].values
    return df_final  


def AVT(dealnum,pctchange,section_return,section_info,volume,d=14,m=28,H=14):
    # d,m,H为参数，风险均值窗口d，收益加权窗口m，H一般为m的一半，作为半衰期
    # dealnum,volume为成交笔数、成交量的原始dataframe
    # pctchange为涨跌幅的原始dataframe
    # section_return为对应行业的涨跌幅的dataframe，行为行业指数，列为日期
    # section_info为股票及其对应的行业的对照表dataframe，列名为'industrycode'
    def weight_omega(m,j,H):
        aa=[2**(-i/H) for i in range(1,m+1)]
        return 2**(-(m-j+1)/H)/sum(aa)

    def risk_today(origin,d):
        return origin.rolling(d).mean()-origin
    
    weight_value=np.array([weight_omega(m,j,H) for j in range(1,m+1)])
    
    stocklist=dealnum.columns.intersection(section_info[section_info['industrycode']!='0'].index)
    dealnum=dealnum[stocklist]
    pctchange=pctchange[stocklist]
    volume=volume[stocklist]
    section_return=section_return.dropna(axis=1, how='any')
    search=section_info.applymap(lambda x:x in section_return.columns)
    selected=section_info[search.industrycode].index
    stocklist=dealnum.columns.intersection(selected)
    dealnum=dealnum[stocklist]
    pctchange=pctchange[stocklist]
    volume=volume[stocklist]
    
    df_exceed=pd.DataFrame(index=pctchange.index,columns=pctchange.columns)
    for ii in pctchange.columns:
        df_exceed[ii]=pctchange[ii]-section_return[section_info.loc[ii].values[0]]
    
    df_avt=volume/dealnum
    df_avt=df_avt**(0.5)
    df_risk=pd.DataFrame(index=pctchange.index,columns=pctchange.columns)
    for ii in pctchange.columns:
        df_risk[ii]=risk_today(df_avt[[ii]],d)
    
    df_riskturnover=df_risk*df_exceed
    df_riskturnover=df_riskturnover.iloc[d:,:]
    df_final=pd.DataFrame(index=df_riskturnover.index[m:],columns=df_riskturnover.columns)
    for jj in range(len(df_final.index)):
        df_final.iloc[jj,:]=weight_value@df_riskturnover.iloc[jj:(jj+m),:].values
    return df_final 

def stdamount(amount,days=20):
    return -amount.rolling(30).std()

def sppercent(ps,days=242):
    return ps.applymap(daoshu).rolling(days).rank()/days

def bppercent(pb,days=242):
    return pb.applymap(daoshu).rolling(days).rank()/days

In [1]:
import statsmodels.api as sm

def _WLS(y, x, weight, intercept = True, res = True):
    '''
    子功能：最小二乘回归求残差、权重，运用于行业、市值中性化处理
    输入参数：
        y:回归应变量，数组形式
        x:回归的自变量，数组形式，长度和y相同F
        weight:回归的权重，数组形式，如没有加权回归，传入1即可
        intercept:是否在回归中考虑截距项，布尔型，默认考虑
        res:是否需要输出残差项,布尔型，默认需要输出
    返回值：
        param:回归系数，数组形式
        （如res=True）残差项：数组形式
    '''
    if intercept:
        x = sm.add_constant(x)
    else:
        x = x   

    model = sm.WLS(y, x, weights=weight)
    result = model.fit()
    params = result.params
    if res:
        return params,y-result.fittedvalues
    else:
        return params,result.rsquared


def cap_neutralize(factor,cap):
# 市值中性化
# 输入factor：因子值的dataframe和cap：市值的dataframe
# 输出中性化后的因子dataframe
    cap=cap.loc[factor.index,factor.columns]
    cap=cap.loc[:, cap.apply(lambda x: (x != 0).all())]
    cap=np.log(cap)
    factor=factor.loc[cap.index,cap.columns]
    result=pd.DataFrame(index=factor.index,columns=factor.columns)
    for day in factor.index:
        yy=factor.loc[day].values
        xx=cap.loc[day].values
        aa,bb=_WLS(yy,xx,1,True,True)
        if np.isnan(aa[0]):
            result.loc[day]=factor.loc[day]
        else:
            result.loc[day]=bb
    return result


# MAD去极值法
def MAD_winsorize(factor,std):
    mf=factor.median(axis=1)
    MAD=factor.apply(lambda x:abs(x-mf)).median(axis=1)
    highbound=mf+std*MAD
    lowbound=mf-std*MAD
    return factor.clip(lowbound,highbound,axis=0)


# 因子标准化
def factor_standarize(factor,methods=1):
    if methods==1:
        MEAN=factor.mean(axis=1)
        STD=factor.std(axis=1)
        re=factor.apply(lambda x:(x-MEAN)/STD)
    elif methods==2:
        MIN=factor.min(axis=1)
        MAX=factor.max(axis=1)
        re=factor.apply(lambda x:(x-MIN)/(MAX-MIN))
    return re

In [7]:
alpha_13 = alpha13(volume,close)
alpha_14 = alpha14(Open,volume,returns)

alpha_16 = alpha16(high,volume)

alpha_18 = alpha18(close,Open)

In [5]:
turnover_14 = turnover14(turnover)
turnover_28 = turnover28(turnover)
turnover_14_predicted = turnover14predicted(turnover_14,turnover_28)
alpha_37 = alpha37(Open,close)
stdamount=stdamount(amount)

In [28]:
alpha_1 = alpha1(close,returns)
alpha_2 = alpha2(Open,close,volume)
alpha_3 = alpha3(Open,volume)
alpha_4 = alpha4(low)
alpha_5 = alpha5(Open,vwap,close)
alpha_6 = alpha6(Open, volume)
alpha_7 = alpha7(volume,close)
alpha_8 = alpha8(Open,returns)
alpha_9 = alpha9(close)
alpha_10 = alpha10(close)
alpha_11 = alpha11(vwap,close,volume)
alpha_12 = alpha12(volume,close)
alpha_13 = alpha13(volume,close)
alpha_14 = alpha14(Open,volume,returns)
alpha_15 = alpha15(high,volume)
alpha_16 = alpha16(high,volume)
alpha_17 = alpha17(volume,close)
alpha_18 = alpha18(close,Open)
alpha_19 = alpha19(close,returns)
alpha_20 = alpha20(Open,high,close,low)
alpha_21 = alpha21(volume,close)
alpha_22 = alpha22(high,volume,close)
alpha_23 = alpha23(high,close)
alpha_24 = alpha24(close)
alpha_25 = alpha25(volume,returns,vwap,high,close)
alpha_26 = alpha26(volume,high)
alpha_27 = alpha27(volume,vwap)
alpha_28 = alpha28(volume,high,low,close)
alpha_29 = alpha29(close,returns)
alpha_30 = alpha30(close,volume)
alpha_31 = alpha31(close,low,volume)
alpha_32 = alpha32(close,vwap)
alpha_33 = alpha33(Open,close)
alpha_34 = alpha34(close,returns)
alpha_35 = alpha35(volume,close,high,low,returns)
alpha_36 = alpha36(Open,close,volume,returns,vwap)
alpha_37 = alpha37(Open,close)
alpha_38 = alpha38(close,Open)
alpha_39 = alpha39(volume,close,returns)
alpha_40 = alpha40(high,volume)
alpha_41 = alpha41(high,low,vwap)
alpha_42 = alpha42(vwap,close)
alpha_43 = alpha43(volume,close)
alpha_44 = alpha44(high,volume)
alpha_45 = alpha45(close,volume)
alpha_46 = alpha46(close)
alpha_47 = alpha47(volume,close,high,vwap)
alpha_49 = alpha49(close)
alpha_50 = alpha50(volume,vwap)
alpha_51 = alpha51(close)
alpha_52 = alpha52(returns,volume,low)
alpha_53 = alpha53(close,high,low)
alpha_54 = alpha54(Open,close,high,low)
alpha_55 = alpha55(high,low,close,volume)
alpha_56 = alpha56(returns,cap)
alpha_57 = alpha57(close,vwap)
alpha_60 = alpha60(close,high,low,volume)
alpha_61 = alpha61(volume,vwap)
alpha_62 = alpha62(volume,high,low,Open,vwap)
alpha_64 = alpha64(high,low,Open,volume,vwap)
alpha_65 = alpha65(volume,vwap,Open)
alpha_66 = alpha66(vwap,low,Open,high)
alpha_68 = alpha68(volume,high,close,low)
alpha_71 = alpha71(volume,close,low,Open,vwap)
alpha_72 = alpha72(volume,high,low,vwap)
alpha_73 = alpha73(vwap,Open,low)
alpha_74 = alpha74(volume,close,high,vwap)
alpha_75 = alpha75(volume,vwap,low)
alpha_77 = alpha77(volume,high,low,vwap)
alpha_78 = alpha78(volume,low,vwap)
alpha_81 = alpha81(volume,vwap)
alpha_83 = alpha83(high,low,close,volume)
alpha_84 = alpha84(vwap,close)
alpha_85 = alpha85(volume,high,close,low)
alpha_86 = alpha86(volume,close,Open,vwap)
alpha_88 = alpha88(volume,Open,low,high,close)
alpha_92 = alpha92(volume,high,low,close,Open)
alpha_94 = alpha94(volume,vwap)
alpha_95 = alpha95(volume,high,low,Open)
alpha_96 = alpha96(volume,vwap,close)
alpha_98 = alpha98(volume,Open,vwap)
alpha_99 = alpha99(volume,high,low)
alpha_101 = alpha101(close,Open,high,low)

<ipython-input-27-fe763ff8c29d>:92: RuntimeWarning: divide by zero encountered in log
  r1 = rank(delta(np.log(volume), 2))


AttributeError: 'DataFrame' object has no attribute 'as_matrix'

In [75]:
alpha_14 = alpha14(Open,volume,returns)

In [58]:
data_alpha =  ['alpha_1','alpha_2','alpha_3','alpha_4','alpha_5','alpha_6','alpha_7','alpha_8','alpha_9','alpha_10',
               'alpha_11','alpha_12','alpha_13','alpha_14','alpha_15','alpha_16','alpha_17','alpha_18','alpha_19','alpha_20',
               'alpha_21','alpha_22','alpha_23','alpha_24','alpha_25','alpha_26','alpha_27','alpha_28','alpha_29','alpha_30',
               'alpha_31','alpha_32','alpha_33','alpha_34','alpha_35','alpha_36','alpha_37','alpha_38','alpha_39','alpha_40',
               'alpha_41','alpha_42','alpha_43','alpha_44','alpha_45','alpha_46','alpha_47','alpha_49','alpha_50',
               'alpha_51','alpha_52','alpha_53','alpha_54','alpha_55','alpha_56','alpha_57','alpha_60',
               'alpha_61','alpha_62','alpha_64','alpha_65','alpha_66','alpha_68',
               'alpha_71','alpha_72','alpha_73','alpha_74','alpha_75','alpha_77','alpha_78',
               'alpha_81','alpha_83','alpha_84','alpha_85','alpha_86','alpha_88',
               'alpha_92','alpha_94','alpha_95','alpha_96','alpha_98','alpha_99','alpha_101','df_turnover14_predicted']

In [33]:
bppercent=bppercent(pb_lf)

In [34]:
factor_list = ['bppercent']
import os
def timestamp_to_str(x):
    x = str(x)
    return x[:4]+x[5:7]+x[8:10]

def save(p):
    close = pd.read_csv("close_hfq.csv",parse_dates=[0], index_col=0).fillna(value=0)

    for i in p:
        name= i
        print(name)
        # if not os.path.isdir('C:/Users/可涵/Desktop/factor_test_0317/%s' % (name)):
        if not os.path.isdir('..\\%s' % (name)):
            os.makedirs('..\\%s' % (name))
        eval(i).index = close.index

        frame_used = eval(i).loc['20220101':'20230412',:]
        st = timestamp_to_str(list(frame_used.index)[0])
        ed = timestamp_to_str(list(frame_used.index)[-1])
        print(st)
        print(ed)

        frame_used.to_excel("..\\%s\\%s_%s_%s.xlsx" % (name,name,st,ed))


if __name__ == '__main__':

    save(factor_list)
        



bppercent
20220104
20230412


In [6]:
#turnoveradvance01测试用
turnover_notdrop  = pd.read_csv("turnover.csv",parse_dates=[0], index_col=0)
returns_notdrop = pd.read_csv("change_ratio.csv",parse_dates=[0], index_col=0)
section=pd.read_csv('industry申万三级.csv', index_col=[0], parse_dates=[0])
shenwan_return=pd.read_excel('申万三级收益率.xlsx', index_col=[0], parse_dates=[0])
aaa=turnoveradvance01(turnover_notdrop,returns_notdrop,shenwan_return,section)

In [8]:
aaa.to_csv('turnover_advance01.csv')

In [9]:
# AVT测试用
transactionAmount_notdrop  = pd.read_csv("transactionAmount.csv",parse_dates=[0], index_col=0)
returns_notdrop = pd.read_csv("change_ratio.csv",parse_dates=[0], index_col=0).fillna(value=0)
volume_notdrop = pd.read_csv("volume.csv", parse_dates=[0], index_col=0).fillna(value=0)
section=pd.read_csv('industry申万三级.csv', index_col=[0], parse_dates=[0])
shenwan_return=pd.read_excel('申万三级收益率.xlsx', index_col=[0], parse_dates=[0])
bbb=AVT(transactionAmount_notdrop,returns_notdrop,shenwan_return,section,volume_notdrop)

In [10]:
bbb.to_csv('AVT.csv')

In [4]:
# #turnoveradvance01测试用
# turnover_notdrop  = pd.read_csv("turnover.csv",parse_dates=[0], index_col=0)
# returns_notdrop = pd.read_csv("change_ratio.csv",parse_dates=[0], index_col=0)
# section=pd.read_csv('industry申万三级.csv', index_col=[0], parse_dates=[0])
# shenwan_return=pd.read_excel('申万三级收益率.xlsx', index_col=[0], parse_dates=[0])
# aaa=turnoveradvance01(turnover_notdrop,returns_notdrop,shenwan_return,section)
# # AVT测试用
# transactionAmount_notdrop  = pd.read_csv("transactionAmount.csv",parse_dates=[0], index_col=0)
# returns_notdrop = pd.read_csv("change_ratio.csv",parse_dates=[0], index_col=0).fillna(value=0)
# volume_notdrop = pd.read_csv("volume.csv", parse_dates=[0], index_col=0).fillna(value=0)
# section=pd.read_csv('industry申万三级.csv', index_col=[0], parse_dates=[0])
# shenwan_return=pd.read_excel('申万三级收益率.xlsx', index_col=[0], parse_dates=[0])
# bbb=AVT(transactionAmount_notdrop,returns_notdrop,shenwan_return,section,volume_notdrop)